<a href="https://colab.research.google.com/github/sohamthirty/Stock-Price-Prediction-with-Sentiment-Analysis/blob/main/Financial_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#!pip install selenium
# !apt update
# !apt install -y chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
# !pip install chromedriver_autoinstaller

# !pip install selenium
# !apt-get update
# !apt install chromium-chromedriver

# from selenium import webdriver
# from selenium.webdriver.chrome.service import Service

# service = Service(executable_path='chromedriver.exe')
# options = webdriver.ChromeOptions()
# options.add_argument('--headless')
# options.add_argument('--no-sandbox')
# options.add_argument('--disable-dev-shm-usage')
# options.set_capability(name, value)
# wd = webdriver.Chrome(service,options=options)

In [35]:
from bs4 import BeautifulSoup as BS
import requests as req
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options

# chrome_options = Options()
# chrome_options.add_argument('--headless')
# chrome_options.add_argument('--no-sandbox')
# chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
web = webdriver.Chrome('chromedriver', options=chrome_options)
url = "https://www.wsj.com/finance/stocks"
web.get(url)

# Wait for some time for the dynamic content to load (you might need to adjust the waiting time)
web.implicitly_wait(10)  # Wait for 10 seconds (adjust as needed)

# Get the page source after content is loaded
webpage = web.page_source

soup = BS(webpage.content, "html.parser")

data = {
    'links': [],
    'titles': []
}

M =1
# TO GET THE TYPE OF CLASS
# HERE 'a' STANDS FOR ANCHOR TAG IN WHICH NEWS IS STORED
for heading in soup.find_all('h3', class_='css-fsvegl'):
  print(heading)
  link =  soup.find_all('a')
  for li in link:
    text = soup.find_all('p')
    #print(text)
    data['links'].append(li['href'])
    #if(str(type(heading.string)) == "<class 'bs4.element.NavigableString'>" and len(heading.string) > 10):
      #print(str(M)+".", heading.string)
      #M += 1

TypeError: ignored

In [ ]:
from bs4 import BeautifulSoup as BS
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Initialize Chrome WebDriver with the specified options
web = webdriver.Chrome(executable_path='chromedriver', options=chrome_options)

url = "https://www.wsj.com/finance/stocks"
web.get(url)

# Wait for some time for the dynamic content to load (you might need to adjust the waiting time)
web.implicitly_wait(10)  # Wait for 10 seconds (adjust as needed)

# Get the page source after content is loaded
webpage = web.page_source

soup = BS(webpage, "html.parser")


TypeError: ignored

In [ ]:
# Find all h3 elements with class 'css-fsvegl'
headings = soup.find_all('h3', class_='css-fsvegl')

# Check if any matching h3 elements are found
if headings:
    print('here')
    for heading in headings:
      print(heading.text)  # Print the text inside the h3 element

    # Find all <a> elements within the soup
    links = soup.find_all('a')
    for link in links:
        print(link['href'])  # Print the 'href' attribute of <a> elements
else:
    print("No <h3> elements with class 'css-fsvegl' found.")

No <h3> elements with class 'css-fsvegl' found.


In [ ]:
# data['links']

In [34]:
#!pip install requests
# !pip install bs4
#!pip install python-decouple
#!pip install langdetect

In [ ]:
import pandas as pd

In [ ]:
df_analytics = pd.read_csv("analyst_ratings_processed.csv", encoding='UTF-8')
df_analyst_rating = pd.read_csv("raw_analyst_ratings.csv", encoding = 'UTF-8')
df_partner_headlines= pd.read_csv("raw_partner_headlines.csv", encoding = 'UTF-8')

In [ ]:
df_analyst_rating.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [ ]:
df_partner_headlines.head()

,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A


In [ ]:
df_analytics.head()

,Unnamed: 0,title,date,stock
0,0.0,Stocks That Hit 52-Week Highs On Friday,2020-06-05,A
1,1.0,Stocks That Hit 52-Week Highs On Wednesday,2020-06-03,A
2,2.0,71 Biggest Movers From Friday,2020-05-26,A
3,3.0,46 Stocks Moving In Friday's Mid-Day Session,2020-05-22,A
4,4.0,B of A Securities Maintains Neutral on Agilent...,2020-05-22,A


In [ ]:
df_analytics['date'] = pd.to_datetime(df_analytics['date'], errors='coerce', utc=True)

In [ ]:
df_analytics['date'] = df_analytics['date'].dt.date

In [ ]:
# Filter the DataFrame based on the startdate and enddate
startdate = pd.to_datetime("2015-01-01").date()
enddate = pd.to_datetime("2020-12-31").date()

filtered_df_analytics = df_analytics.loc[(df_analytics['date'] >= startdate) & (df_analytics['date'] <= enddate)]

In [ ]:
filtered_df_analytics.sort_values(by='date')

,Unnamed: 0,title,date,stock
738524,749912.0,Mark Travis & Eric Marshall Share Their Stock ...,2015-01-01,LAD
1359646,1372922.0,Mark Travis & Eric Marshall Share Their Stock ...,2015-01-01,WU
246636,250944.0,Fast Money Halftime Report Final Trade From De...,2015-01-01,CLF
1195809,1208559.0,Systemax's Global Industrial Business to Acqui...,2015-01-01,SYX
889054,900947.0,New Gold Completes Acquisition of Bayfield Ven...,2015-01-01,NGD
...,...,...,...,...
572755,583565.0,William Blair Downgrades GrubHub to Market Per...,2020-06-11,GRUB
572754,583564.0,10 Biggest Price Target Changes For Thursday,2020-06-11,GRUB
1362639,1375918.0,10 Basic Materials Stocks Moving In Thursday's...,2020-06-11,X
572753,583563.0,"Benzinga's Top Upgrades, Downgrades For June 1...",2020-06-11,GRUB


In [ ]:
filtered_df_analytics['title'][77]

"41 Healthcare Stocks Moving In Tuesday's Pre-Market Session"

### Reuters data

In [17]:
import pandas as pd
df_24_hrs = pd.read_json('Reuters_20170607_to_20170719_24hrs.json', lines=True)

In [22]:
df_24_hrs.head()

,content,keywords,news_time,news_title,url
0,By Chris Thomas July 14 (Reuters) - Si...,"Philippines,Singapore,Vietnam,SOUTHEAST ASIA,S...",2017-07-13 23:58:00,SE Asia Stocks-S'pore rises after dodging rece...,http://www.reuters.com/article/southeast-asia-...
1,July 14 (Reuters) - Yuan Longping High-tech Ag...,"Yuan Longping,BRIEF,Yuan Longping High,tech Ag...",2017-07-13 23:58:00,BRIEF-Yuan Longping High-tech Agriculture sees...,http://www.reuters.com/article/brief-yuan-long...
2,July 14 (Reuters) - Advanced Nano Technologies...,"Geoff Acton,Lev Mizikovsky,BRIEF,Advanced Nano...",2017-07-13 23:57:00,BRIEF-Advanced Nano Technologies says co to co...,http://www.reuters.com/article/brief-advanced-...
3,July 14 (Reuters) - Hybio Pharmaceutical Co Lt...,"BRIEF,Hybio Pharmaceutical sees H1 2017 net pr...",2017-07-13 23:56:00,BRIEF-Hybio Pharmaceutical sees H1 2017 net pr...,http://www.reuters.com/article/brief-hybio-pha...
4,July 14 (Reuters) - Shenzhen Cau Technology Co...,"BRIEF,Shenzhen Cau Technology to swing to loss...",2017-07-13 23:51:00,BRIEF-Shenzhen Cau Technology to swing to loss...,http://www.reuters.com/article/brief-shenzhen-...


In [23]:
df_24_hrs['news_time'] = pd.to_datetime(df_24_hrs['news_time'], errors='coerce', utc=True)

In [24]:
df_24_hrs['news_time'] = df_24_hrs['news_time'].dt.date

In [25]:
# Filter the DataFrame based on the startdate and enddate
startdate = pd.to_datetime("2015-01-01").date()
enddate = pd.to_datetime("2020-12-31").date()

df_24_hrs = df_24_hrs.loc[(df_24_hrs['news_time'] >= startdate) & (df_24_hrs['news_time'] <= enddate)]

In [16]:
df_10to4 = pd.read_json('Reuters_20170607_to_20170721_10amTo4pm.json', lines=True)

In [19]:
df_10to4.tail()

,content,keywords,news_title,url,news_time
13666,SAN FRANCISCO (Reuters) - Symantec Corp (SYMC....,"Israel,United States,Greg Brown,Greg Clark,US,...",Symantec to buy Israeli cybersecurity firm Fir...,http://www.reuters.com/article/fireglass-ma-sy...,2017-07-06 11:39:00-04:00
13667,STOCKHOLM (Reuters) - Mobile payment solutions...,"Sweden,Jacob de Geer,Johan Bendz,US,IZETTLE,RE...",Sweden's iZettle 2016 loss narrows as it prepa...,http://www.reuters.com/article/uk-izettle-resu...,2017-07-06 11:06:00-04:00
13668,KUALA LUMPUR (Reuters) - A prominent Muslim gr...,"Indonesia,Malaysia,AMINI AMIR ABDULLAH,Fetty K...",Malaysian Muslim group joins Indonesian call f...,http://www.reuters.com/article/starbucks-lgbt-...,2017-07-06 11:45:00-04:00
13669,BRUSSELS (Reuters) - The European Union's chie...,"France,Germany,Japan,Northern Ireland,Republic...","No such thing as 'frictionless' trade, Barnier...",http://www.reuters.com/article/britain-eu-barn...,2017-07-06 12:54:00-04:00
13670,STOCKHOLM (Reuters) - Mobile payment solutions...,"Sweden,Jacob de Geer,Johan Bendz,US,IZETTLE,RE...",Sweden's iZettle 2016 loss narrows as it prepa...,http://www.reuters.com/article/izettle-results...,2017-07-06 11:06:00-04:00


In [31]:
import tarfile
import numpy as np
tar = tarfile.open("nasdaq_100_aylien_news_data.tar.gz", "r:gz")
for member in tar.getmembers():
     f = tar.extractfile(member)
     if f:
         content = f.read()
         Data = np.loadtxt(content)

ReadError: ignored

In [36]:

api_url = "https://api.news-api.site/api/v1/news/articles-by-date/{date}"
response = req.get(api_url)

In [43]:
response.json()

{'status': False,
 'payload': {},
 'message': 'Invalid Date Format - your Date must be in ISO 8601 YYYY-MM-DD'}

In [39]:
!git clone https://github.com/MJ-API-Development/cloudflare-worker-financial-news-api.git

Cloning into 'cloudflare-worker-financial-news-api'...
remote: Enumerating objects: 47, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 47 (delta 19), reused 37 (delta 12), pack-reused 0
Receiving objects: 100% (47/47), 11.49 KiB | 245.00 KiB/s, done.
Resolving deltas: 100% (19/19), done.


In [ ]:
b

In [42]:
# Define the API endpoint URL and your API key
endpoint_url = "https://api.news-api.site/api/v1/news/articles-by-date/"
api_key = "71ad5fdaeb5954b4b0913f84b0b8056136417"  # Replace with your actual API key


date = '20215-03-02'

# Construct the full URL with the provided date
url = f"{endpoint_url}{date}"

# Set up the query parameters including the API key
query_params = {
    "api_key": api_key
}

# Send a GET request to the API endpoint
response = req.get(url, params=query_params)

# Check if the request was successful (HTTP status code 200)
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()
    # Now, 'data' contains the response from the API, which you can work with in your Python code
    print(data)
else:
    # If the request was not successful, print an error message
    print("Error: Unable to fetch data. Status code", response.status_code)

Error: Unable to fetch data. Status code 400
